In [55]:
import os
os.chdir('/home/data/yangz/bioinfo/packages/exprmat/exprmat/data/hsa/assemblies/grch37')
os.getcwd()

'/home/data/yangz/bioinfo/packages/exprmat/exprmat/data/hsa/assemblies/grch37'

In [57]:
root = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.25_GRCh37.p13/"

In [ ]:
!wget -O gaps.tsv.gz {root}GCF_000001405.25_GRCh37.p13_genomic_gaps.txt.gz
!gunzip gaps.tsv.gz
!wget -O patches.tsv {root}GCF_000001405.25_GRCh37.p13_assembly_regions.txt
!wget -O regions.tsv {root}GCF_000001405.25_GRCh37.p13_assembly_report.txt

In [59]:
import pandas as pd

In [64]:
gaps = pd.read_table('gaps.tsv', comment = '#', sep = '\t', header = None)
patches = pd.read_table('patches.tsv', comment = '#', sep = '\t', header = None)
regions = pd.read_table('regions.tsv', comment = '#', sep = '\t', header = None)

In [65]:
gaps.columns = ['.seqid', 'start', 'end', 'length', 'type', 'linkage.evidence']
gaps

,.seqid,start,end,length,type,linkage.evidence
0,NT_113907.1,23054,23153,100,unknown,unspecified
1,NT_113950.2,9934,10033,100,unknown,unspecified
2,NT_167244.1,207808,252060,44253,within_scaffold,clone_contig
3,NT_167244.1,327075,356458,29384,within_scaffold,clone_contig
4,NT_167244.1,448575,536172,87598,within_scaffold,clone_contig
...,...,...,...,...,...,...
508,NC_000024.9,22369680,22419679,50000,within_scaffold,clone_contig
509,NC_000024.9,23901429,23951428,50000,between_scaffolds,na
510,NC_000024.9,28819362,58819361,30000000,heterochromatin,na
511,NC_000024.9,58917657,58967656,50000,between_scaffolds,na


In [66]:
patches.columns = ['name', 'chr', 'start', 'end', 'type', 'genbank', 'refseq', 'unit']
patches

,name,chr,start,end,type,genbank,refseq,unit
0,REGION77,1,3823584.0,3845268.0,fix-patch,JH806574.2,NW_004070864.2,PATCHES
1,REGION60,1,31872759.0,32017063.0,fix-patch,GL949741.1,NW_003571030.1,PATCHES
2,1P11,1,120471050.0,121346350.0,fix-patch,JH636053.3,NW_003871056.3,PATCHES
3,1Q21,1,143871003.0,150440213.0,fix-patch,JH636052.4,NW_003871055.3,PATCHES
4,REGION2,1,153673007.0,153838214.0,novel-patch,GL383518.1,NW_003315905.1,PATCHES
...,...,...,...,...,...,...,...,...
212,REGION97,X,142299066.0,150174083.0,fix-patch,JH806600.2,NW_004070890.2,PATCHES
213,REGION67,X,151734490.0,154906585.0,fix-patch,JH159150.3,NW_003871103.3,PATCHES
214,PAR-2,X,154931044.0,155260560.0,PAR,na,na,na
215,PAR-1,Y,10001.0,2649520.0,PAR,na,na,na


In [67]:
regions.columns = [
    'name', 'type', 'chr', 'type.molecule', 'genbank', 'relation', 'refseq', 'unit', 'length', 'ucsc'
]

regions.iloc[:, 1:]

,type,chr,type.molecule,genbank,relation,refseq,unit,length,ucsc
0,assembled-molecule,1,Chromosome,CM000663.1,=,NC_000001.10,Primary Assembly,249250621,chr1
1,assembled-molecule,2,Chromosome,CM000664.1,=,NC_000002.11,Primary Assembly,243199373,chr2
2,assembled-molecule,3,Chromosome,CM000665.1,=,NC_000003.11,Primary Assembly,198022430,chr3
3,assembled-molecule,4,Chromosome,CM000666.1,=,NC_000004.11,Primary Assembly,191154276,chr4
4,assembled-molecule,5,Chromosome,CM000667.1,=,NC_000005.9,Primary Assembly,180915260,chr5
...,...,...,...,...,...,...,...,...,...
292,alt-scaffold,6,Chromosome,GL000255.1,=,NT_167248.1,ALT_REF_LOCI_6,4611984,chr6_qbl_hap6
293,alt-scaffold,6,Chromosome,GL000256.1,=,NT_167249.1,ALT_REF_LOCI_7,4928567,chr6_ssto_hap7
294,alt-scaffold,4,Chromosome,GL000257.1,=,NT_167250.1,ALT_REF_LOCI_8,590426,chr4_ctg9_hap1
295,alt-scaffold,17,Chromosome,GL000258.1,=,NT_167251.1,ALT_REF_LOCI_9,1680828,chr17_ctg5_hap1


In [68]:
regions['type'].value_counts()

type
fix-patch               131
novel-patch              73
unplaced-scaffold        39
assembled-molecule       25
unlocalized-scaffold     20
alt-scaffold              9
Name: count, dtype: int64

In [69]:
patches['type'].value_counts()

type
fix-patch       131
novel-patch      73
alt-scaffold      9
PAR               3
Name: count, dtype: int64

We will organize the genomic fragments into chromosomes (with alternative scaffolds, fix patches and 
novel patches and gaps included). This will produce a comprehensive mapping to the assembly.

### Retrieve primary regions

In [70]:
primary = regions.loc[regions['type'] == 'assembled-molecule', :].copy()
primary['start'] = 0
primary['end'] = primary['length']
primary['length.map'] = primary['length']
primary['type'] = 'primary'
primary['subtype'] = primary['type.molecule'].str.lower()
del primary['type.molecule']
primary = primary.rename(columns = {'length': 'length.real'})
del primary['relation']
primary

,name,type,chr,genbank,refseq,unit,length.real,ucsc,start,end,length.map,subtype
0,1,primary,1,CM000663.1,NC_000001.10,Primary Assembly,249250621,chr1,0,249250621,249250621,chromosome
1,2,primary,2,CM000664.1,NC_000002.11,Primary Assembly,243199373,chr2,0,243199373,243199373,chromosome
2,3,primary,3,CM000665.1,NC_000003.11,Primary Assembly,198022430,chr3,0,198022430,198022430,chromosome
3,4,primary,4,CM000666.1,NC_000004.11,Primary Assembly,191154276,chr4,0,191154276,191154276,chromosome
4,5,primary,5,CM000667.1,NC_000005.9,Primary Assembly,180915260,chr5,0,180915260,180915260,chromosome
5,6,primary,6,CM000668.1,NC_000006.11,Primary Assembly,171115067,chr6,0,171115067,171115067,chromosome
6,7,primary,7,CM000669.1,NC_000007.13,Primary Assembly,159138663,chr7,0,159138663,159138663,chromosome
7,8,primary,8,CM000670.1,NC_000008.10,Primary Assembly,146364022,chr8,0,146364022,146364022,chromosome
8,9,primary,9,CM000671.1,NC_000009.11,Primary Assembly,141213431,chr9,0,141213431,141213431,chromosome
9,10,primary,10,CM000672.1,NC_000010.10,Primary Assembly,135534747,chr10,0,135534747,135534747,chromosome


### Patches and alternative scaffolds

In [71]:
aligned = patches[['name', 'chr', 'start', 'end', 'genbank', 'refseq', 'unit', 'type']] \
    .join(regions[['genbank', 'length']].set_index('genbank'), on = 'genbank', how = 'left')
aligned = aligned.loc[[x in primary.chr.tolist() for x in aligned['chr']]]
aligned['length.real'] = aligned['length']
aligned['length.map'] = aligned['end'] - aligned['start']
aligned['type'] = [x.lower() for x in aligned['type']]
# aligned['subtype'] = aligned['type']
del aligned['length']

In [72]:
aligned

,name,chr,start,end,genbank,refseq,unit,type,length.real,length.map
0,REGION77,1,3823584.0,3845268.0,JH806574.2,NW_004070864.2,PATCHES,fix-patch,22982.0,21684.0
1,REGION60,1,31872759.0,32017063.0,GL949741.1,NW_003571030.1,PATCHES,fix-patch,151551.0,144304.0
2,1P11,1,120471050.0,121346350.0,JH636053.3,NW_003871056.3,PATCHES,fix-patch,1676126.0,875300.0
3,1Q21,1,143871003.0,150440213.0,JH636052.4,NW_003871055.3,PATCHES,fix-patch,7283150.0,6569210.0
4,REGION2,1,153673007.0,153838214.0,GL383518.1,NW_003315905.1,PATCHES,novel-patch,182439.0,165207.0
...,...,...,...,...,...,...,...,...,...,...
212,REGION97,X,142299066.0,150174083.0,JH806600.2,NW_004070890.2,PATCHES,fix-patch,6530008.0,7875017.0
213,REGION67,X,151734490.0,154906585.0,JH159150.3,NW_003871103.3,PATCHES,fix-patch,3110903.0,3172095.0
214,PAR-2,X,154931044.0,155260560.0,na,na,na,par,NaN,329516.0
215,PAR-1,Y,10001.0,2649520.0,na,na,na,par,NaN,2639519.0


### Gaps

In [73]:
gps = gaps.join(primary[['refseq', 'chr', 'ucsc']].set_index('refseq'), on = '.seqid', how = 'left')
gps = gps[~ gps.chr.isna()]
gps['name'] = ['gap:' + str(i + 1) for i in range(len(gps))]
gps['subtype'] = [x.replace('_', '-') for x in gps['type']]
gps['type'] = 'gap'
del gps['linkage.evidence']
gps = gps.rename(columns = {'.seqid': 'refseq'})
gps['length.real'] = gps['length']
gps['length.map'] = gps['length']
del gps['length']

In [74]:
gps

,refseq,start,end,type,chr,ucsc,name,subtype,length.real,length.map
143,NC_000001.10,1,10000,gap,1,chr1,gap:1,telomere,10000,10000
144,NC_000001.10,177418,227417,gap,1,chr1,gap:2,within-scaffold,50000,50000
145,NC_000001.10,267720,317719,gap,1,chr1,gap:3,between-scaffolds,50000,50000
146,NC_000001.10,471369,521368,gap,1,chr1,gap:4,between-scaffolds,50000,50000
147,NC_000001.10,2634221,2684220,gap,1,chr1,gap:5,within-scaffold,50000,50000
...,...,...,...,...,...,...,...,...,...,...
508,NC_000024.9,22369680,22419679,gap,Y,chrY,gap:366,within-scaffold,50000,50000
509,NC_000024.9,23901429,23951428,gap,Y,chrY,gap:367,between-scaffolds,50000,50000
510,NC_000024.9,28819362,58819361,gap,Y,chrY,gap:368,heterochromatin,30000000,30000000
511,NC_000024.9,58917657,58967656,gap,Y,chrY,gap:369,between-scaffolds,50000,50000


### Merge

In [75]:
conc = pd.concat([primary, aligned, gps])
conc['ucsc'] = ['chrM' if x == 'MT' else 'chr' + x for x in conc['chr']]
conc['genbank'] = [x if str(x) != 'nan' else 'na' for x in conc['genbank']]
conc['unit'] = [x if str(x) != 'nan' else 'na' for x in conc['unit']]
conc['length.real'] = [x if str(x) != 'nan' else y for x, y in zip(conc['length.real'], conc['length.map'])]
conc['subtype'] = [x if str(x) != 'nan' else y for x, y in zip(conc['subtype'], conc['type'])]
conc['length.map'] = conc['length.map'].astype('int')
conc['start'] = conc['start'].astype('int')
conc['end'] = conc['end'].astype('int')
conc = conc.sort_values(['chr', 'start', 'length.map'])

In [80]:
conc.to_csv('architecture.tsv', sep = '\t', index = False)

In [77]:
sizes = conc.loc[conc.type == 'primary', ['ucsc', 'length.map']].set_index('ucsc').to_dict()['length.map']

In [78]:
sizes

{'chr1': 249250621,
 'chr10': 135534747,
 'chr11': 135006516,
 'chr12': 133851895,
 'chr13': 115169878,
 'chr14': 107349540,
 'chr15': 102531392,
 'chr16': 90354753,
 'chr17': 81195210,
 'chr18': 78077248,
 'chr19': 59128983,
 'chr2': 243199373,
 'chr20': 63025520,
 'chr21': 48129895,
 'chr22': 51304566,
 'chr3': 198022430,
 'chr4': 191154276,
 'chr5': 180915260,
 'chr6': 171115067,
 'chr7': 159138663,
 'chr8': 146364022,
 'chr9': 141213431,
 'chrM': 16569,
 'chrX': 155270560,
 'chrY': 59373566}

In [79]:
import json
with open('size.json', 'w') as f:
    json.dump(sizes, f, indent = 4)

In [81]:
!rm gaps.tsv patches.tsv regions.tsv